In [ ]:
#| default_exp protein_intensity_estimation

In [ ]:
%reload_ext autoreload

%autoreload 2

# Protein Intenstiity Estimation

This notebook tests the protein LFQ intensity estimation step. It is in principle a wrapper around the functionality of the normalization.py class, which is used to shift precursors or transition intensity traces on top of each other.

# Unit Tests

#### Classes for testcase generation

In [ ]:
import pandas as pd
import numpy as np
import directlfq.protein_intensity_estimation as lfq_protint
#test df cutting

def test_sorting_by_num_nans():
    vals1 = np.array([9, np.nan, np.nan, np.nan])
    vals2 = np.array([5, 6, np.nan, np.nan])
    vals3 = np.array([1, 2, 3,np.nan ])

    df = pd.DataFrame([vals1, vals2, vals3],index=[['P', 'P', 'P'],['A', 'B', 'C']])
    pcutter = lfq_protint.ProtvalCutter(df,maximum_df_length=2)
    sorted_idx = pcutter._sorted_idx
    df_sorted = df.loc[sorted_idx]
    
    assert np.allclose(df_sorted.iloc[2].to_numpy(), vals1,equal_nan=True)
    assert np.allclose(df_sorted.iloc[0].to_numpy(), vals3,equal_nan=True)
    

def test_cutting_of_df():
    vals1 = np.array([9, np.nan, np.nan, np.nan])
    vals2 = np.array([5, 6, np.nan, np.nan])
    vals3 = np.array([1, 2, 3,np.nan ])

    df = pd.DataFrame([vals1, vals2, vals3],index=[['A', 'B', 'C']])
    pcutter = lfq_protint.ProtvalCutter(df, maximum_df_length=2)
    cut_df = pcutter.get_dataframe()
    ion_idx = [x[0] for x in cut_df.index]
    print(ion_idx)
    assert ion_idx == ['C', 'B']


test_sorting_by_num_nans()
test_cutting_of_df()


['C', 'B']


In [ ]:
import directlfq.normalization as lfq_norm
import directlfq.test_utils as lfq_testutils

def test_that_profiles_without_noise_are_shifted_exactly_on_top_of_each_other():
    peptide1= lfq_testutils.PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=3000, add_noise=False)
    peptide2= lfq_testutils.PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0.9, systematic_peptide_shift=3, add_noise=False)
    peptide3= lfq_testutils.PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=0.1, add_noise=False)
    peptide4= lfq_testutils.PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0.9, systematic_peptide_shift=100, add_noise=False)
    protein_df = lfq_testutils.ProteinProfileGenerator([peptide1, peptide2, peptide3, peptide4]).protein_profile_dataframe
    display(protein_df)
    normed_ion_profile = lfq_norm.normalize_ion_profiles(protein_df)
    display(normed_ion_profile)
    column_from_shifted = normed_ion_profile.iloc[:,11].dropna().to_numpy()
    display(column_from_shifted)
    assert np.allclose(column_from_shifted, column_from_shifted[0])

def test_that_profiles_with_noise_are_close():
    peptide1= lfq_testutils.PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=3000, add_noise=True)
    peptide2= lfq_testutils.PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=3, add_noise=True)
    peptide3= lfq_testutils.PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=0.1, add_noise=True)
    peptide4= lfq_testutils.PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)

    protein_df = lfq_testutils.ProteinProfileGenerator([peptide1, peptide2, peptide3, peptide4]).protein_profile_dataframe
    display(protein_df)
    
    normed_ion_profile = lfq_norm.normalize_ion_profiles(protein_df)
    display(normed_ion_profile)
    column_from_shifted = normed_ion_profile.iloc[:,9].dropna().to_numpy()

    assert np.allclose(column_from_shifted, column_from_shifted[0],rtol=0.01, atol=0.01)

test_that_profiles_without_noise_are_shifted_exactly_on_top_of_each_other()
test_that_profiles_with_noise_are_close()


0          1          2          3          4          5   \
protein ion                                                                    
protA   0         NaN  44.474241  41.489085  43.216307  43.505708  42.338404   
        1         NaN        NaN        NaN        NaN        NaN        NaN   
        2    29.73029  29.601566  26.616410  28.343632  28.633033        NaN   
        3         NaN        NaN        NaN        NaN  38.598817        NaN   

                    6          7          8          9          10         11  \
protein ion                                                                     
protA   0    43.454211  43.102621  41.897534  43.625508  43.261340  40.203349   
        1          NaN        NaN        NaN        NaN        NaN        NaN   
        2    28.581537  28.229946  27.024859  28.752833  28.388665  25.330674   
        3          NaN        NaN        NaN        NaN        NaN        NaN   

                    12         13         14         15         16         17  \
protein ion                                                                     
protA   0    44.447364  44.554027  44.692343  44.701690  43.106703  44.630975   
        1    34.481579        NaN        NaN        NaN        NaN  34.665191   
        2    29.574689        NaN  29.819668  29.829015  28.234029  29.758301   
        3    39.540473        NaN        NaN        NaN        NaN        NaN   

                    18         19  
protein ion                        
protA   0    43.380650        NaN  
        1          NaN        NaN  
        2    28.507975  28.534739  
        3          NaN        NaN

0          1          2          3          4          5   \
protein ion                                                                     
protA   0          NaN  44.474241  41.489085  43.216307  43.505708  42.338404   
        1          NaN        NaN        NaN        NaN        NaN        NaN   
        2    44.602965  44.474241  41.489085  43.216307  43.505708        NaN   
        3          NaN        NaN        NaN        NaN  43.505708        NaN   

                    6          7          8          9         10         11  \
protein ion                                                                    
protA   0    43.454211  43.102621  41.897534  43.625508  43.26134  40.203349   
        1          NaN        NaN        NaN        NaN       NaN        NaN   
        2    43.454211  43.102621  41.897534  43.625508  43.26134  40.203349   
        3          NaN        NaN        NaN        NaN       NaN        NaN   

                    12         13         14        15         16         17  \
protein ion                                                                    
protA   0    44.447364  44.554027  44.692343  44.70169  43.106703  44.630975   
        1    44.447364        NaN        NaN       NaN        NaN  44.630975   
        2    44.447364        NaN  44.692343  44.70169  43.106703  44.630975   
        3    44.447364        NaN        NaN       NaN        NaN        NaN   

                   18         19  
protein ion                       
protA   0    43.38065        NaN  
        1         NaN        NaN  
        2    43.38065  43.407413  
        3         NaN        NaN

array([40.20334853, 40.20334853])

0          1          2          3          4          5   \
protein ion                                                                     
protA   0    44.602965  44.474240  41.489085  43.216306  43.505708  42.338405   
        1    34.637191  34.508449  31.523327  33.250533  33.539906  32.372622   
        2    29.730299  29.601596  26.616364  28.343634  28.633164  27.465805   
        3    39.696074  39.567350  36.582194  38.309420  38.598821  37.431511   

                    6          7          8          9          10         11  \
protein ion                                                                     
protA   0    43.454211  43.102621  41.897533  43.625508  43.261340  40.203346   
        1    33.488420  33.136837  31.931740  33.659734  33.295572  30.237537   
        2    28.581529  28.229941  27.024871  28.752936  28.388685  25.330857   
        3    38.547322  38.195732  36.990644  38.718614  38.354449  35.296454   

                    12         13         14         15         16         17  \
protein ion                                                                     
protA   0    44.447364  44.554027  44.692343  44.701690  43.106703  44.630976   
        1    34.481578  34.588245  34.726571  34.735917  33.140931  34.665203   
        2    29.574649  29.681455  29.819767  29.828994  28.234112  29.758290   
        3    39.540472  39.647136  39.785450  39.794799  38.199810  39.724082   

                    18         19  
protein ion                        
protA   0    43.380650  43.407413  
        1    33.414868  33.441634  
        2    28.508008  28.534752  
        3    38.473762  38.500524

0          1          2          3          4          5   \
protein ion                                                                     
protA   0    44.602965  44.474240  41.489085  43.216306  43.505708  42.338405   
        1    44.602971  44.474230  41.489107  43.216313  43.505686  42.338402   
        2    44.602957  44.474255  41.489023  43.216293  43.505823  42.338464   
        3    44.602965  44.474240  41.489085  43.216310  43.505712  42.338402   

                    6          7          8          9          10         11  \
protein ion                                                                     
protA   0    43.454211  43.102621  41.897533  43.625508  43.261340  40.203346   
        1    43.454200  43.102617  41.897521  43.625514  43.261352  40.203317   
        2    43.454188  43.102600  41.897529  43.625595  43.261344  40.203515   
        3    43.454213  43.102623  41.897535  43.625505  43.261340  40.203344   

                    12         13         14         15         16         17  \
protein ion                                                                     
protA   0    44.447364  44.554027  44.692343  44.701690  43.106703  44.630976   
        1    44.447358  44.554025  44.692351  44.701697  43.106711  44.630983   
        2    44.447308  44.554114  44.692426  44.701652  43.106771  44.630949   
        3    44.447363  44.554027  44.692341  44.701690  43.106701  44.630973   

                    18         19  
protein ion                        
protA   0    43.380650  43.407413  
        1    43.380648  43.407414  
        2    43.380667  43.407411  
        3    43.380653  43.407414

In [ ]:
import directlfq.protein_intensity_estimation as intensity_estimation
import directlfq.test_utils as lfq_testutils

def test_that_protein_intensities_are_retained():
    peptide1= lfq_testutils.PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=3000, add_noise=True)
    peptide2= lfq_testutils.PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=3, add_noise=True)
    peptide3= lfq_testutils.PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=0.1, add_noise=True)
    peptide4= lfq_testutils.PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    
    peptide_profiles = [peptide1, peptide2, peptide3, peptide4]
    summed_intensity_protein = sum([np.nansum(x.peptide_profile_vector) for x in peptide_profiles])
    
    protein_df = lfq_testutils.ProteinProfileGenerator([peptide1, peptide2, peptide3, peptide4]).protein_profile_dataframe
    protein_df_normed, _ = intensity_estimation.estimate_protein_intensities(protein_df, min_nonan=1, num_samples_quadratic=100, num_cores=1)
    display(protein_df_normed)
    display(protein_df_normed.iloc[0,1:].to_numpy())
    summed_lfq_intensities = np.sum(protein_df_normed.iloc[0,1:].to_numpy())
    assert np.allclose(summed_lfq_intensities, summed_intensity_protein)

test_that_protein_intensities_are_retained()



2023-12-08 12:22:20,257 - directlfq.protein_intensity_estimation - INFO - 1 prots total
2023-12-08 12:22:20,258 - directlfq.protein_intensity_estimation - INFO - prot 0


,protein,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,protA,2.534980e+13,2.318589e+13,2.928181e+12,9.695020e+12,1.184861e+13,5.275621e+12,1.143313e+13,8.960380e+12,3.886496e+12,...,1.000241e+13,1.201040e+12,2.275799e+13,2.450432e+13,2.696994e+13,2.714526e+13,8.985770e+12,2.584682e+13,1.086479e+13,1.106823e+13


array([25349795642833.24, 23185890220888.97, 2928180586448.17,
       9695020357645.807, 11848609977072.47, 5275621107950.519,
       11433132923005.893, 8960379724734.42, 3886495984197.8735,
       12874540472742.57, 10002405236736.053, 1201039877802.1846,
       22757993614330.746, 24504324707272.664, 26969944908322.53,
       27145258441058.31, 8985769914714.105, 25846821137849.547,
       10864793434866.52, 11068226808633.043], dtype=object)

In [ ]:
import directlfq.protein_intensity_estimation as intensity_estimation
import directlfq.test_utils as lfq_testutils

def run_with_multiple_proteins():
    peptide1= lfq_testutils.PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0.1, systematic_peptide_shift=3000, add_noise=True)
    peptide2= lfq_testutils.PeptideProfile(protein_name="protA",fraction_zeros_in_profile=0, systematic_peptide_shift=3, add_noise=True)
    peptide3= lfq_testutils.PeptideProfile(protein_name="protA", fraction_zeros_in_profile=0, systematic_peptide_shift=0.1, add_noise=True)
    peptide4= lfq_testutils.PeptideProfile(protein_name="protB",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide5= lfq_testutils.PeptideProfile(protein_name="protC",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide6= lfq_testutils.PeptideProfile(protein_name="protD",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide7= lfq_testutils.PeptideProfile(protein_name="protD",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)
    peptide8= lfq_testutils.PeptideProfile(protein_name="protD",fraction_zeros_in_profile=0, systematic_peptide_shift=100, add_noise=True)

    peptide_profiles = [peptide1, peptide2, peptide3, peptide4, peptide5, peptide6, peptide7, peptide8]
    protein_df = lfq_testutils.ProteinProfileGenerator(peptide_profiles).protein_profile_dataframe
    protein_df_normed, _ = intensity_estimation.estimate_protein_intensities(protein_df, min_nonan=1, num_samples_quadratic=100, num_cores=1)
    display(protein_df_normed)
    assert len(protein_df_normed.index) == 4

run_with_multiple_proteins()



2023-12-08 12:26:12,022 - directlfq.protein_intensity_estimation - INFO - 4 prots total
2023-12-08 12:26:12,024 - directlfq.protein_intensity_estimation - INFO - prot 0


,protein,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,protA,2.531384e+13,2.315286e+13,2.924049e+12,9.681216e+12,1.183173e+13,5.268128e+12,1.141686e+13,8.947613e+12,3.880908e+12,...,9.988645e+12,1.199318e+12,2.272559e+13,2.446956e+13,2.693163e+13,2.710675e+13,8.972944e+12,2.580999e+13,1.084931e+13,1.105253e+13
1,protB,8.906532e+11,8.146271e+11,1.028814e+11,3.406308e+11,4.162962e+11,1.853563e+11,4.016973e+11,3.148190e+11,1.365508e+11,...,3.514311e+11,4.219781e+10,7.995918e+11,8.609472e+11,9.475768e+11,9.537363e+11,3.157105e+11,9.081169e+11,3.817282e+11,3.888771e+11
2,protC,8.906544e+11,8.146285e+11,1.028819e+11,3.406296e+11,4.162955e+11,1.853560e+11,4.016979e+11,3.148195e+11,1.365506e+11,...,3.514307e+11,4.219777e+10,7.995936e+11,8.609486e+11,9.475759e+11,9.537344e+11,3.157112e+11,9.081153e+11,3.817297e+11,3.888760e+11
3,protD,2.671963e+12,2.443884e+12,3.086451e+11,1.021892e+12,1.248886e+12,5.560711e+11,1.205096e+12,9.444586e+11,4.096517e+11,...,1.054292e+12,1.265943e+11,2.398774e+12,2.582846e+12,2.842730e+12,2.861209e+12,9.471320e+11,2.724343e+12,1.145188e+12,1.166632e+12


## Learning tests

In [ ]:
import pandas as pd
import numpy as np

def test_that_dataframe_is_generated_as_expected():
    vals1 = np.array([1, 2, 3,4 ])
    vals2 = np.array([5, 6, 7, 8])
    vals3 = np.array([9, 10, 11, 12])
    df = pd.DataFrame([vals1, vals2, vals3],index=['A', 'A', 'A'])
    display(df)
    assert df.iloc[2, 2] == 11
    assert df.iloc[1, 2] == 7




In [ ]:

test_that_dataframe_is_generated_as_expected()

,0,1,2,3
A,1,2,3,4
A,5,6,7,8
A,9,10,11,12


In [ ]:
def test_retrieval_of_numpy_arrays_from_dataframe():
    vals1 = np.array([1, 2, 3,4 ])
    vals2 = np.array([5, 6, 7, 8])
    vals3 = np.array([9, 10, 11, 12])
    df = pd.DataFrame([vals1, vals2, vals3],index=[['A', 'B', 'C'], ['a', 'b', 'a']])
    display(df)
    assert np.allclose(vals2, df.loc['B'])
    assert np.allclose([2, 6, 10], df.loc[:,1])


In [ ]:

test_retrieval_of_numpy_arrays_from_dataframe()

,,0,1,2,3
A,a,1,2,3,4
B,b,5,6,7,8
C,a,9,10,11,12


In [ ]:
def test_setting_numpy_seed():
    from numpy.random import MT19937
    from numpy.random import RandomState, SeedSequence

    rs = RandomState(MT19937(SeedSequence(42)))
    res = rs.randint(10,size=20)
    display(res)


In [ ]:

test_setting_numpy_seed()

array([2, 6, 8, 8, 3, 3, 3, 3, 4, 7, 2, 7, 5, 4, 0, 8, 1, 3, 7, 1])